# Scrapeo wikipedia para sacar info de los artistas


In [1]:
import time

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as bs

from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import threading

from concurrent.futures import ThreadPoolExecutor
from selenium.common.exceptions import NoSuchElementException

import pandas as pd



In [2]:
PATH = 'driver/chromedriver.exe'

driver = webdriver.Chrome(PATH) 

# Scrap de wikipedia para un solo artista

In [3]:
url = 'https://es.wikipedia.org/wiki/Madonna'
driver.get(url)

In [4]:
driver.find_elements(By.CSS_SELECTOR, '#mw-content-text > div.mw-parser-output > table.infobox.biography.vcard')[0].text.split('\n')[4:8]
driver.quit()

# Creo funcion para sacar datos especificos de artistas

In [5]:
def info_artistas(artista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    driver.get(f'https://es.wikipedia.org/wiki/{artista}')

    
    nacimiento_formacion = None
    nacionalidad_origen = None

    try:
        info = driver.find_elements(By.CSS_SELECTOR, 'table.infobox')[0].text.split('\n')
        for data in info: 
            if 'Nacimiento' in data or 'Formación' in data:
                nacimiento_formacion = data
            elif 'Nacionalidad' in data or 'Origen' in data:
                nacionalidad_origen = data.split(":")[-1].strip()
    except NoSuchElementException:
        print(f'No se encontró la info para el artista: {artista}')
    
    driver.quit()

    return nacimiento_formacion, nacionalidad_origen



In [6]:
def fetch_info(artista): 
    return info_artistas(artista, PATH)

# Creo la tabla Artistas


In [7]:
artistas = pd.read_csv('../database/csv_limpios/albums.csv')

In [8]:
artistas = artistas.drop(columns= ['Year', 'Ranking','Album','Worldwide Sales','CDs','Tracks'])


In [9]:
lista_artistas = artistas['Artist'].tolist()


In [10]:
#artistas['Info']= artistas['Artist'].apply(info_artistas)

In [11]:
#artistas.to_csv('../database/csv_limpios/pasolargo_info.csv', index=False)

In [12]:
with ThreadPoolExecutor(max_workers=8) as executor: 
    #Map the `fetch_info` function to each artist
    results = list(executor.map(fetch_info, lista_artistas))

In [13]:
artistas['Info'] = results

In [15]:
artistas = artistas.reset_index().rename(columns={'index': 'artistas_id'})

In [17]:
#Separo la columna info en dos columnas
artistas['Nacimiento_formacion'] = artistas['Info'].apply(lambda x: x[0])
artistas['Nacionalidad_origen'] = artistas['Info'].apply(lambda x: x[1])

In [18]:
artistas.fillna('Desconocido', inplace=True)

In [19]:
artistas = artistas.drop(columns=['Info'])

In [20]:
artistas.head()

,artistas_id,Artist,Genre,Nacimiento_formacion,Nacionalidad_origen
0,0,Madonna,Pop,Nacimiento 16 de agosto de 1958 (65 años),Nacionalidad Estadounidense
1,1,New_Kids_on_the_Block,Pop,Desconocido,"Origen Boston, Estados Unidos"
2,2,Garth Brooks,Country,Nacimiento 7 de febrero de 1962 (61 años),Nacionalidad Estadounidense
3,3,Metallica,Rock,Desconocido,"Origen Los Ángeles, California,"
4,4,Nirvana_(banda),Rock,Desconocido,"Origen Aberdeen, Washington,"


# Export la tabla artistas


In [21]:
artistas.to_csv('../database/csv_limpios/artistas.csv',encoding='latin1', index=False)